In [1]:
# Configuración inicial del proyecto
import importlib
import sys
import os

# Reload the quick_setup module to pick up changes
if 'ePy_docs.api.quick_setup' in sys.modules:
    importlib.reload(sys.modules['ePy_docs.api.quick_setup'])

from ePy_docs.api.quick_setup import quick_setup

# Setup completo con layout corporativo, sincronización activada
result = quick_setup(layout='corporate', sync_files=True)

# Configurar directorio para reportes en results/report
current_dir = os.getcwd()
reports_dir = os.path.join(current_dir, "results", "report")
os.makedirs(reports_dir, exist_ok=True)

# Configurar el writer para usar este directorio
writer.output_dir = reports_dir

print(f"✅ Setup completado - Writer disponible globalmente")
print(f"📁 Reports will be saved to: {reports_dir}")

📁 Directory structure created: data, data/configuration, data/examples, data/user/brand, data/user/templates, data\configuration\components, data\configuration\core, data\configuration\files, data\configuration\project, data\configuration\units, data\robot, results, results/figures, results/report, results/tables, results\figures, results\tables
🎯 ePy_docs library initialized:
   📋 Layout: corporate
   🔄 Config sync: ✅ Enabled
   ⚖️ Responsibility page: ❌ Disabled
   📋 Project setup: ✅ Loaded from setup.json
   📊 CSV defaults: ✅ Configured
   📝 Quarto formats: ✅ Configured
⚠️  Units system initialization failed: 'units'
🔢 Units system ready - Global functions available: convert(), convert_to_default_length(), convert_to_default_force()
📝 Report writer ready - Global variable available: writer
📁 File manager ready - Global variable available: files
✅ All systems ready!
✅ Setup completado - Writer disponible globalmente
📁 Reports will be saved to: c:\Users\ingah\estructuraPy\ePy_docs\res

Traceback (most recent call last):
  File "C:\Users\ingah\estructuraPy\ePy_docs\src\ePy_docs\api\quick_setup.py", line 252, in _setup_units_system
    units_config = builtins.configs['units']
                   ~~~~~~~~~~~~~~~~^^^^^^^^^
KeyError: 'units'


In [2]:
# Importaciones para análisis
from ePy_analysis.robot.nodes import RobotNodes, RobotSupports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from ePy_docs.core.setup import get_output_directories

print("✅ Importaciones completadas")

✅ Importaciones completadas


In [3]:
# Configuración de rutas de datos
data_dir = os.path.join(current_dir, "data", "robot")

# Rutas de archivos CSV
nodes_csv_path = os.path.join(data_dir, "nodes.csv")
reactions_csv_path = os.path.join(data_dir, "reactions.csv") 
combinations_csv_path = os.path.join(data_dir, "combinations.csv")
blocks_csv_path = os.path.join(current_dir, "data", "user", "rigid_blocks.csv")

# Crear directorios de resultados
results_dir = os.path.join(current_dir, "results")
os.makedirs(results_dir, exist_ok=True)
os.makedirs(os.path.join(results_dir, "tables"), exist_ok=True)
os.makedirs(os.path.join(results_dir, "figures"), exist_ok=True)

# Verificar existencia de archivos
files_to_check = {
    'blocks_csv': blocks_csv_path,
    'nodes_csv': nodes_csv_path,
    'reactions_csv': reactions_csv_path,
    'combinations_csv': combinations_csv_path
}

for name, path in files_to_check.items():
    exists = os.path.exists(path)
    print(f"{name}: {exists}")

print(f"✅ Configuración de rutas completada")
print(f"📁 Directorio de datos: {data_dir}")
print(f"📁 Directorio de resultados: {results_dir}")

blocks_csv: True
nodes_csv: True
reactions_csv: True
combinations_csv: True
✅ Configuración de rutas completada
📁 Directorio de datos: c:\Users\ingah\estructuraPy\ePy_docs\data\robot
📁 Directorio de resultados: c:\Users\ingah\estructuraPy\ePy_docs\results


In [4]:
# 4. Inicializar datos con archivos CSV de ROBOT (con manejo de encoding)
print("🔄 Initializing data and configuration...")

# Función para leer CSV con diferentes encodings y delimitadores
def safe_read_csv(file_path):
    encodings = ['utf-16le', 'utf-16', 'utf-8', 'latin1', 'cp1252']
    delimiters = [';', ',', '\t']
    
    for encoding in encodings:
        for delimiter in delimiters:
            try:
                df = pd.read_csv(file_path, encoding=encoding, sep=delimiter, decimal=',')
                if len(df.columns) > 1:  # Verificar que se leyó correctamente
                    print(f"✅ Successfully read {file_path} with encoding={encoding}, delimiter={delimiter}")
                    return df
            except:
                continue
    
    print(f"❌ Failed to read {file_path} with any encoding/delimiter combination")
    return None

# Usar DataFrames ya cargados o leer desde archivo
if 'nodes_df' not in globals():
    try:
        nodes_df = safe_read_csv(nodes_csv_path)
        if nodes_df is None:
            # Fallback: crear datos de ejemplo
            nodes_df = pd.DataFrame({
                'Node': [1, 2, 3, 4, 5],
                'X (cm)': [0, 500, 1000, 1500, 2000],
                'Y (cm)': [0, 0, 0, 0, 0],
                'Z (cm)': [0, 0, 0, 0, 0]
            })
            print("📊 Using sample nodes data")
    except Exception as e:
        print(f"Error reading nodes: {e}")
        nodes_df = pd.DataFrame({
            'Node': [1, 2, 3, 4, 5],
            'X (cm)': [0, 500, 1000, 1500, 2000],
            'Y (cm)': [0, 0, 0, 0, 0],
            'Z (cm)': [0, 0, 0, 0, 0]
        })
        
if 'reactions_df' not in globals():
    try:
        reactions_df = safe_read_csv(reactions_csv_path)
        if reactions_df is None:
            # Fallback: crear datos de ejemplo
            reactions_df = pd.DataFrame({
                'Node': [1, 2, 3, 4, 5],
                'FX (kgf)': [100, 200, 150, 300, 250],
                'FY (kgf)': [0, 0, 0, 0, 0],
                'FZ (kgf)': [500, 800, 600, 1200, 900],
                'MX (kgf·m)': [0, 0, 0, 0, 0],
                'MY (kgf·m)': [10, 20, 15, 30, 25],
                'MZ (kgf·m)': [0, 0, 0, 0, 0]
            })
            print("📊 Using sample reactions data")
    except Exception as e:
        print(f"Error reading reactions: {e}")
        reactions_df = pd.DataFrame({
            'Node': [1, 2, 3, 4, 5],
            'FX (kgf)': [100, 200, 150, 300, 250],
            'FY (kgf)': [0, 0, 0, 0, 0],
            'FZ (kgf)': [500, 800, 600, 1200, 900],
            'MX (kgf·m)': [0, 0, 0, 0, 0],
            'MY (kgf·m)': [10, 20, 15, 30, 25],
            'MZ (kgf·m)': [0, 0, 0, 0, 0]
        })
        
if 'support_nodes_df' not in globals():
    support_nodes_df = reactions_df[reactions_df['FZ (kgf)'].abs() > 0].copy()

print(f"✅ Data available: {len(nodes_df)} nodes, {len(reactions_df)} reactions")

# Configurar unidades del proyecto
units_config = {
    'length': 'cm',
    'force': 'kgf', 
    'moment': 'kgf·cm'
}

# Variables de unidad
length_unit = units_config['length'] 
force_unit = units_config['force']
moment_unit = units_config['moment']
area_unit = f"{length_unit}²"
volume_unit = f"{length_unit}³"

# Configuración del reporte (definir aquí para evitar NameError)
report_config = {
    'title': 'Foundation Analysis Report',
    'warning_text': 'Generated report - verify all calculations',
    'warning_title': 'Important Notice',
    'nodes_section_title': 'Nodes and Supports Data',
    'nodes_description': 'Node coordinates and support information:',
    'nodes_table_title': 'Node coordinates',
    'supports_table_title': 'Support nodes',
    'supports_colored_table_title': 'Support nodes with reactions',
    'supports_description': 'Support information:',
    'reactions_summary_title': 'Reactions summary',
    'sample_table_title': 'Sample node coordinates',
    'sample_size': 5,
    'max_rows_per_table': [20, 20, 20],
    'max_reaction_rows': 5,
    'table_palette': 'engineering'
}

print("✅ Configuration and data initialization complete")

🔄 Initializing data and configuration...
✅ Successfully read c:\Users\ingah\estructuraPy\ePy_docs\data\robot\nodes.csv with encoding=utf-16le, delimiter=;
✅ Successfully read c:\Users\ingah\estructuraPy\ePy_docs\data\robot\reactions.csv with encoding=utf-8, delimiter=;
✅ Data available: 60 nodes, 189 reactions
✅ Configuration and data initialization complete


# Foundation Analysis Report

Este notebook demuestra un análisis integral de fundaciones usando la metodología de bloques rígidos, siguiendo el Código Geotécnico de Costa Rica (CGCR-2024).

## Características:
- ✅ Layout corporativo con sincronización activada
- ✅ Análisis con datos de ROBOT Structural Analysis  
- ✅ Generación de reportes en formatos QMD, HTML, PDF
- ✅ Archivos guardados en `results/report`

In [5]:
# Test de generacion de reportes
import shutil

# Verificar y corregir directorio de salida
print("Current writer output_dir:", writer.output_dir)

# Forzar configuracion del directorio correcto
reports_dir = os.path.join(current_dir, "results", "report")
writer.output_dir = reports_dir
print("Writer output_dir updated to:", writer.output_dir)

# Mover archivos existentes de results a results/report
results_main = os.path.join(current_dir, "results")
if os.path.exists(results_main):
    for file in ['report.pdf', 'report.html', 'report.qmd']:
        src = os.path.join(results_main, file)
        dst = os.path.join(reports_dir, file)
        if os.path.exists(src):
            shutil.move(src, dst)
            print("Moved", file, "to results/report/")
    
    # Copiar directorios de tablas y figuras
    for dir_name in ['tables', 'figures']:
        src_dir = os.path.join(results_main, dir_name)
        dst_dir = os.path.join(reports_dir, dir_name)
        if os.path.exists(src_dir):
            if os.path.exists(dst_dir):
                shutil.rmtree(dst_dir)
            shutil.copytree(src_dir, dst_dir)
            print("Copied", dir_name, "to results/report/")

# Agregar contenido basico al reporte
writer.add_h1("Foundation Analysis Report")
writer.add_text("Este reporte demuestra un analisis integral de fundaciones usando la metodologia de bloques rigidos.")

# Tabla de ejemplo con datos del analisis
sample_data = pd.DataFrame({
    'Node': [1, 2, 3, 4, 5],
    'X (cm)': [0, 500, 1000, 1500, 2000],
    'Y (cm)': [0, 0, 0, 0, 0],
    'Force (kgf)': [100, 200, 150, 300, 250]
})

writer.add_table(sample_data, title="Datos estructurales de ejemplo")

# Generar reportes en results/report
print("Generating reports in:", writer.output_dir)

# Generar solo HTML y QMD por ahora para evitar errores de PDF
try:
    result = writer.generate(qmd=True, pdf=False, html=True)
    print("QMD and HTML generated successfully")
    
    # Intentar PDF
    result = writer.generate(qmd=False, pdf=True, html=False)
    print("PDF generated successfully")
except Exception as e:
    print("PDF generation failed:", str(e))
    print("QMD and HTML generated successfully")

# Verificar archivos generados
if os.path.exists(writer.output_dir):
    files = [f for f in os.listdir(writer.output_dir) if f.endswith(('.pdf', '.html', '.qmd'))]
    print("Report files in results/report:", files)
    
    # Verificar subdirectorios
    for subdir in ['tables', 'figures']:
        subdir_path = os.path.join(writer.output_dir, subdir)
        if os.path.exists(subdir_path):
            files_in_subdir = len(os.listdir(subdir_path))
            print(subdir + " contains", files_in_subdir, "files")
else:
    print("Output directory not found")

print("Test completado - Todos los archivos se guardan en results/report")

Current writer output_dir: c:\Users\ingah\estructuraPy\ePy_docs\results\report
Writer output_dir updated to: c:\Users\ingah\estructuraPy\ePy_docs\results\report
Copied tables to results/report/
Copied figures to results/report/


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\ingah\\estructuraPy\\ePy_docs\\data/configuration\\components\\report.json'

## 3. Report content

In [ ]:
writer.add_h1(report_config.get('title', 'Foundation Analysis Report 1'))

writer.add_chunk(
    f"""Length unit: ${length_unit}$
Area unit: ${area_unit}$
Volume unit: ${volume_unit}$
Force unit: ${force_unit}$
Moment unit: ${moment_unit}$"""
)

writer.add_content(f"""
                   
# Units

## Units of Measurement

### Units again
                   
Length unit: ${length_unit}$

Area unit: ${area_unit}$

Volume unit: ${volume_unit}$

Force unit: ${force_unit}$

Moment unit: ${moment_unit}$
"""
)

writer.add_chunk(
"""
x = 2
y = 5
z = x * y
print(f'The result is: {z}')
"""
)


writer.add_chunk_executable(
"""
x = 2
y = 5
z = x * y
print(f'The result is: {z}')
""",
language='python'
)

writer.add_content(
    f"""Length unit: ${length_unit}$
Area unit: ${area_unit}$
Volume unit: ${volume_unit}$
Force unit: ${force_unit}$
Moment unit: ${moment_unit}$"""
)

writer.add_tip(
    f"""📏 Length unit: ${length_unit}$
    📐 Area unit: ${area_unit}$
    📦 Volume unit: ${volume_unit}$
    🏋️ Force unit: ${force_unit}$
    ⚖️ Moment unit: ${moment_unit}$
""", "prueba con emojis")

writer.add_tip(
    f"""📏 Length unit: ${length_unit}$
    📐 Area unit: ${area_unit}$
    📦 Volume unit: ${volume_unit}$
    🏋️ Force unit: ${force_unit}$
    ⚖️ Moment unit: ${moment_unit}$
""", "prueba con emojis")

writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_tip(
    f"""Length unit: ${length_unit}$
    Area unit: ${area_unit}$
    Volume unit: ${volume_unit}$
    Force unit: ${force_unit}$
    Moment unit: ${moment_unit}$
""", "prueba sin emojis")

writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_warning(
    f"""Length unit: ${length_unit}$
Area unit: ${area_unit}$
Volume unit: ${volume_unit}$
Force unit: ${force_unit}$
Moment unit: ${moment_unit}$
""", "prueba sin emojis ni espacios")

writer.add_tip(
    report_config.get('warning_text', 'Generated report - verify all calculations'),
    report_config.get('warning_title', 'Important Notice')
    )

writer.add_h1(report_config.get('title', 'Foundation Analysis Report'))

writer.add_success(
    f"""Length unit: $${length_unit}$$
Area unit: $${area_unit}$$
Volume unit: $${volume_unit}$$
Force unit: $${force_unit}$$
Moment unit: $${moment_unit}$$
""", "prueba sin emojis ni espacios")

writer.add_warning(
    report_config.get('warning_text', 'Generated report - verify all calculations'),
    report_config.get('warning_title', 'Important Notice')
    )

# 📋 AGREGAR CONTENIDO ADICIONAL AL INFORME
# Aprovechando las capacidades de ePy_suite para enriquecer el PDF

# 1. Agregar análisis de conversión de unidades
writer.add_h2("Unit Conversion Analysis")
writer.add_content("""
This section demonstrates the unit conversion capabilities integrated within the project. 
The ePy_suite system provides comprehensive support for engineering unit conversions, 
which is crucial for international projects and multi-system calculations.
                   
```{python}
x = 2 + 2
print(f'{x} m')
```
""")

# Crear datos de ejemplo para conversiones de unidades
from ePy_docs.units.converter import UnitConverter

# Usar el converter ya inicializado
force_conversions = []
moment_conversions = []
stress_conversions = []

# Ejemplos de conversiones de fuerzas
force_examples = [
    (1000, 'kgf', 'kN'),
    (500, 'kgf', 'lbf'),
    (10, 'kN', 'tonf'),
    (2500, 'lbf', 'kgf'),
    (100, 'ksi', 'kgf/cm2'),
    (1, 'tonf', 'kN'),
    (2500, 'kgf/cm2', 'psi'),
    (7030.70, 'kgf/cm2', 'ksi'),
    (703, 'MPa', 'ksi'),
    (70, 'MPa', 'kgf/cm2'),
    (1000, 'N', 'kN'),
    (500, 'N', 'lbf'),
    (25, 'N', 'dyn'),
    (10000, 'Pa', 'kgf/m2'),
    (1, 'atm', 'bar'),
    (2.5, 'bar', 'psi'),
    (600, 'psi', 'MPa'),
    (200, 'kgf', 'N'),
    (50, 'kgf', 'dyn'),
    (3500, 'N·m', 'kgf·m'),
    (500, 'N·m', 'lbf·ft'),
    (703, 'MPa', 'bar'),
]

for value, from_unit, to_unit in force_examples:
    try:
        result = converter.universal_unit_converter(value, from_unit, to_unit)
        if isinstance(result, (int, float)):
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result:.2f} {to_unit}",
                'Conversion Factor': f"1 {from_unit} = {result/value:.6f} {to_unit}"
            })
        else:
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result} {to_unit}",
                'Conversion Factor': "See conversion"
            })
    except Exception as e:
        force_conversions.append({
            'Original Value': f"{value} {from_unit}",
            'Converted Value': "Conversion Error",
            'Conversion Factor': f"Error: {str(e)}"
        })

# Crear DataFrame y agregar al reporte
if force_conversions:
    force_df = pd.DataFrame(force_conversions)
    writer.add_h3("Force Unit Conversions")
    writer.add_content("Examples of force unit conversions commonly used in structural analysis:")
    writer.add_table(force_df, title="Force unit conversion examples")

writer.add_h2("Design Recommendations and Best Practices")
writer.add_content("""
Based on the analysis performed, the following recommendations are provided:

### Structural Design Recommendations:
1. **Foundation Sizing**: All calculated foundations should be reviewed for constructability constraints
2. **Load Distribution**: Consider load redistribution for heavily loaded nodes
3. **Construction Tolerances**: Include appropriate tolerances in final dimensions
4. **Material Specifications**: Ensure concrete strength meets or exceeds design requirements

### Quality Control Measures:
                   
$$F = ma$$ {#eq-newton2}
                   
$M + 2 = 0$
                   
1. **Dimensional Verification**: Verify (@eq-newton2) all foundation dimensions before construction
2. **Reinforcement Detailing**: Ensure proper reinforcement placement and cover
3. **Concrete Quality**: Implement proper concrete curing and quality control procedures
4. **Load Testing**: Consider load testing for critical [@CSCR2010_14] foundations

### Maintenance Considerations:
1. **Inspection Schedule**: Establish regular inspection intervals
2. **Settlement Monitoring**: Monitor foundation settlements over time
3. **Drainage Maintenance**: Ensure proper drainage around foundations
4. **Documentation**: Maintain as-built drawings and inspection records
""")

# 4. Agregar información técnica adicional usando las capacidades de conversión
writer.add_h2("Technical Specifications and Conversions")

# 5. Agregar análisis de comparación de métodos
writer.add_h2("Methodology Comparison")
writer.add_warning("""
                   
### Foundation Analysis Methods Comparison

The rigid block method used in this @tbl-6 analysis offers several advantages:

**Advantages:**
- Simple and conservative approach
- Well-established in engineering practice
- Suitable for preliminary design phases
- Compatible with various soil types

**Limitations:**
- May be overly conservative for some applications
- Does not account for soil-structure interaction effects
- Limited consideration of dynamic loading conditions

**Alternative Methods:**
- Finite Element Analysis (FEA) for complex geometries
- Spring-based models for dynamic analysis
- Advanced bearing capacity theories for special conditions
                   


""")


### Nodes and supports

In [ ]:
figure_path = r"C:\Users\ingah\estructuraPy\Examples\data\user\brand\imagotipo.png"
writer.add_image(figure_path, label="imagotipo")
writer.add_image(r"C:\Users\ingah\estructuraPy\Examples\data\user\brand\imagotipo.png", caption="Example image caption")

In [ ]:
# Sistema listo para generar reportes

In [ ]:
writer.add_content("(@eq-newton)")

In [ ]:
writer.add_h2(report_config.get('nodes_section_title', 'Nodes and Supports Data'))
writer.add_content(report_config.get('nodes_description', 'Node coordinates and support information:'))

### Add "elegant" table with nodes coordinates

In [ ]:
writer.add_table(
    nodes_df, 
    title=report_config.get('nodes_table_title', 'Node coordinates'),
    max_rows_per_table=report_config.get('max_rows_per_table', [20, 20, 20])
)

### Add "elegant" table with supports coordinates

In [ ]:
writer.add_content(report_config.get('supports_description', 'Support information:'))
writer.add_table(
    support_nodes_df, 
    title=report_config.get('supports_table_title', 'Support nodes')
)

### Add Colored Table with Nodes Coordinates

In [ ]:
writer.add_colored_table(
    support_nodes_df, 
    title=report_config.get('supports_colored_table_title', 'Support nodes with reactions'),
    palette_name=report_config.get('table_palette', 'engineering'),
    highlight_columns=[f'X ({length_unit})', f'Z ({length_unit})', 'Support']
)

### Table subsets

#### Subset of columns

In [ ]:
writer.add_colored_table(
    nodes_df.head(report_config.get('sample_size', 5)), 
    title=report_config.get('sample_table_title', 'Sample node coordinates'),
    highlight_columns=[f'X ({length_unit})', f'Y ({length_unit})', f'Z ({length_unit})', 'Support']
)

#### Subset of rows

In [ ]:
writer.add_colored_table(
    nodes_df.head(report_config.get('sample_size', 5)),
    title=report_config.get('filtered_sample_title', 'Filtered sample coordinates'),
    highlight_columns=[f'Y ({length_unit})', f'X ({length_unit})'], 
    hide_columns=report_config.get('hidden_columns', [f'Z ({length_unit})', 'support'])
)

#### Subset of rows + columns

In [ ]:
writer.add_colored_table(
    nodes_df,
    title=report_config.get('subset_table_title', 'Subset coordinates'),
    highlight_columns=[f'Y ({length_unit})', f'X ({length_unit})'],
    hide_columns=report_config.get('hidden_columns', [f'Z ({length_unit})', 'support']),
    filter_by=('Node', report_config.get('filter_nodes', ['1', '2', '3']))
)

## Sort by

In [ ]:
reactions_df

In [ ]:
writer.add_colored_table(
    reactions_df, 
    title=report_config.get('reactions_table_title', 'Support reactions'),
    palette_name=report_config.get('table_palette', 'environmental'),
    highlight_columns=[f'FX ({force_unit})', f'FZ ({force_unit})', f'MX ({moment_unit})'],
    sort_by=(f'FZ ({force_unit})', report_config.get('sort_order', 'desc')),
    n_rows=5
)

In [ ]:
writer.add_colored_table(
    reactions_df.head(report_config.get('max_reaction_rows', 5)),
    title=report_config.get('reactions_summary_title', 'Reactions summary'),
    highlight_columns=[f'FX ({force_unit})', f'FZ ({force_unit})', f'MY ({moment_unit})']
)

## 5. Enhanced DCR Analysis with Unit Conversion

In [ ]:
# 📋 AGREGAR CONTENIDO ADICIONAL AL INFORME
# Aprovechando las capacidades de ePy_suite para enriquecer el PDF

# 1. Agregar análisis de conversión de unidades
writer.add_h2("Unit Conversion Analysis")
writer.add_content("""
This section demonstrates the unit conversion capabilities integrated within the project. 
The ePy_suite system provides comprehensive support for engineering unit conversions, 
which is crucial for international projects and multi-system calculations.
""")

# Crear datos de ejemplo para conversiones de unidades
from ePy_docs.units.converter import UnitConverter

# Usar el converter ya inicializado
force_conversions = []
moment_conversions = []
stress_conversions = []

# Ejemplos de conversiones de fuerzas
force_examples = [
    (1000, 'kgf', 'kN'),
    (500, 'kgf', 'lbf'),
    (10, 'kN', 'tonf'),
    (2500, 'lbf', 'kgf'),
    (100, 'ksi', 'kgf/cm2'),
    (1, 'tonf', 'kN'),
    (2500, 'kgf/cm2', 'psi'),
    (7030.70, 'kgf/cm2', 'ksi'),
    (703, 'MPa', 'ksi'),
    (70, 'MPa', 'kgf/cm2'),
    (1000, 'N', 'kN'),
    (500, 'N', 'lbf'),
    (25, 'N', 'dyn'),
    (10000, 'Pa', 'kgf/m2'),
    (1, 'atm', 'bar'),
    (2.5, 'bar', 'psi'),
    (600, 'psi', 'MPa'),
    (200, 'kgf', 'N'),
    (50, 'kgf', 'dyn'),
    (3500, 'N·m', 'kgf·m'),
    (500, 'N·m', 'lbf·ft'),
    (703, 'MPa', 'bar'),
    (1000, 'N', 'kN'),
]

for value, from_unit, to_unit in force_examples:
    try:
        result = converter.universal_unit_converter(value, from_unit, to_unit)
        if isinstance(result, (int, float)):
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result:.2f} {to_unit}",
                'Conversion Factor': f"1 {from_unit} = {result/value:.6f} {to_unit}"
            })
        else:
            force_conversions.append({
                'Original Value': f"{value} {from_unit}",
                'Converted Value': f"{result} {to_unit}",
                'Conversion Factor': "See conversion"
            })
    except Exception as e:
        force_conversions.append({
            'Original Value': f"{value} {from_unit}",
            'Converted Value': "Conversion Error",
            'Conversion Factor': f"Error: {str(e)}"
        })

# Crear DataFrame y agregar al reporte
if force_conversions:
    force_df = pd.DataFrame(force_conversions)
    writer.add_h3("Force Unit Conversions")
    writer.add_content("Examples of force unit conversions commonly used in structural analysis:")
    writer.add_table(force_df, title="Force unit conversion examples")

writer.add_h2("Design Recommendations and Best Practices")
writer.add_content("""
Based on the analysis performed, the following recommendations are provided:

### Structural Design Recommendations:
1. **Foundation Sizing**: All calculated foundations should be reviewed for constructability constraints
2. **Load Distribution**: Consider load redistribution for heavily loaded nodes
3. **Construction Tolerances**: Include appropriate tolerances in final dimensions
4. **Material Specifications**: Ensure concrete strength meets or exceeds design requirements

### Quality Control Measures:
                   
$$
F = m \cdot a
$$ {#eq-newton}
                   
$M + 2 = 0$
                   
1. **Dimensional Verification**: Verify (@eq-newton) all foundation dimensions before construction
2. **Reinforcement Detailing**: Ensure proper reinforcement placement and cover
3. **Concrete Quality**: Implement proper concrete curing and quality control procedures
4. **Load Testing**: Consider load testing for critical [@CSCR2010_14] foundations

### Maintenance Considerations:
1. **Inspection Schedule**: Establish regular inspection intervals
2. **Settlement Monitoring**: Monitor foundation settlements over time
3. **Drainage Maintenance**: Ensure proper drainage around foundations
4. **Documentation**: Maintain as-built drawings and inspection records
""")

# 4. Agregar información técnica adicional usando las capacidades de conversión
writer.add_h2("Technical Specifications and Conversions")

# 5. Agregar análisis de comparación de métodos
writer.add_h2("Methodology Comparison")
writer.add_warning("""
                   
### Foundation Analysis Methods Comparison

The rigid block method used in this @tbl-6 analysis offers several advantages:

**Advantages:**
- Simple and conservative approach
- Well-established in engineering practice
- Suitable for preliminary design phases
- Compatible with various soil types

**Limitations:**
- May be overly conservative for some applications
- Does not account for soil-structure interaction effects
- Limited consideration of dynamic loading conditions

**Alternative Methods:**
- Finite Element Analysis (FEA) for complex geometries
- Spring-based models for dynamic analysis
- Advanced bearing capacity theories for special conditions
""")

writer.generate(pdf=True, markdown=True, html=True)

In [ ]:
writer.add_tip(""" 

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool. 
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                
### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).   

### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                
#### Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
       
""", title="Prueba final")

writer.add_tip(""" 

## Prueba 
                
This report was generated using ePy Suite, a Python-based engineering analysis and design tool.
ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).

## Prueba 
                
* This report was generated using ePy Suite, a **Python-based** engineering analysis and design tool.
* ePy Suite is designed to facilitate structural analysis, foundation design, and other engineering tasks.
* It provides a comprehensive framework for managing engineering projects, including unit conversions, data handling, and
For more information, visit [ePy Suite Documentation](https://epy-suite.readthedocs.io).
                   
### Prueba 
                


                   
""")


In [ ]:
writer.add_quarto_file(r"C:\Users\ingah\estructuraPy\Examples\data\user\document\00_preliminary\front_page.qmd")
writer.add_quarto_file(r"C:\Users\ingah\estructuraPy\Examples\data\user\document\01_introduction\introduction.qmd")
writer.add_quarto_file(r"C:\Users\ingah\estructuraPy\Examples\data\user\document\02_methods\methodology.qmd")
writer.add_quarto_file(r"C:\Users\ingah\estructuraPy\Examples\data\user\document\03_geotech\rockfill.qmd")

In [ ]:
writer.add_markdown_file(r"C:\Users\ingah\estructuraPy\Examples\data\user\Report example.md")


writer.generate(pdf=True, html=True, citation_style='apa')

In [ ]:
writer.generate(qmd=True, html=True, pdf=True)